In [2]:
import pandas as pd
import glob
import plotly.graph_objects as go
import numpy as np


# %%
files = glob.glob('../results_data/multiple_seed/*')
labels = ['pix2pix', 'pix2pix L1', 'pix2pix L1+reset', 'pix2pix patches', 'cycleGAN', 'cycleGAN patches']
file_names = ['test_pix2pix_PAS2IF_van_patches', 'test_pix2pix_PAS2IF_L1_patches', 'test_pix2pix_PAS2IF_L1_reset_patches', '', 'test_cycle_PAS2IF_van_patches', '']
metrics = ['SSIM', 'MAE', 'MSE', 'FID', 'Dice podo. object', 'Dice podo. pixel', 'Dice glom. pixel']

In [1]:
# Read Multi Seeds
df_final = pd.DataFrame()
for file in files:
    if 'Hamburg' in file:
        df_temp = pd.read_excel(file)
        df_temp = df_temp[['Network dice object', 'Network dice pixel']]
        df = pd.DataFrame()
        cell_type = 'glom.' if 'glom' in file else 'podo.'
        df['value'] = df_temp['Network dice object'].append(df_temp['Network dice pixel'])
        df['name'] = 'pix2pix'
        df['metric'] = np.concatenate([np.repeat(f'Dice {cell_type} object',len(df_temp)),
                           np.repeat(f'Dice {cell_type} pixel', len(df_temp))])
    else:
        name = file.split('/')[3].split(' ')[0]
        metric = file.split(' ')[1].split('.')[0]
        df = pd.read_csv(file)
        df = df.filter(regex=r' (?!.*?(MIN|MAX)).*', axis=1)
        df['value'] = df.mean(axis=1)
        df['metric'] = metric
        df['name'] = name
    df_final = pd.concat([df_final, df[['name', 'metric', 'value']]])


NameError: name 'pd' is not defined

In [31]:
## Read Channel data
files = glob.glob('../results_data/channels/*')

df_channels = pd.DataFrame()
for file in files:
        name = file.split('/')[3].split(' ')[0]
        metric = file.split(' ')[3].split('.')[0]
        df = pd.read_csv(file)
        df = df.filter(regex=r' (?!.*?(MIN|MAX)).*', axis=1)
        df['value'] = df.iloc[:, 0]
        df['metric'] = metric
        df['name'] = name
        df_channels = pd.concat([df_channels, df[['name', 'metric', 'value']]])

metrics_channels = ['SSIM', '0', '1', '2']
label_channels = ['all', 'DAPI', 'WT1', 'DACH1']
for label, metric in zip(label_channels, metrics_channels):
    df_channels = df_channels.replace({metric: label})

df_channels

,name,metric,value
0,test_pix2pix_PAS2IF_L1_patches,DACH1,0.610306
1,test_pix2pix_PAS2IF_L1_patches,DACH1,0.662766
2,test_pix2pix_PAS2IF_L1_patches,DACH1,0.646876
3,test_pix2pix_PAS2IF_L1_patches,DACH1,0.596551
4,test_pix2pix_PAS2IF_L1_patches,DACH1,0.646989
...,...,...,...
47,test_pix2pix_PAS2IF_L1_patches,all,0.783263
48,test_pix2pix_PAS2IF_L1_patches,all,0.755221
49,test_pix2pix_PAS2IF_L1_patches,all,0.718438
50,test_pix2pix_PAS2IF_L1_patches,all,0.775916


In [91]:
for label, file_name in zip(labels, file_names):
    df_final = df_final.replace({file_name: label})

In [18]:
def boxplot(df: pd.DataFrame, metric: str, labels:list, export=False) -> None:
    df_plot = df[df.metric == metric]
    colors = ['#4038E8', '#3A63F2', '#408ADB', '#3AC4F2', '#30F541', '#3EF595']
    fig = go.Figure()
    for col, name in zip(colors, labels):
        fig.add_trace(go.Box(y=df_plot['value'][df_plot.name == name],
            name=name, marker_color=col, boxmean=True, showlegend=False, boxpoints='all'
        ))
    fig.update_layout(template='simple_white', yaxis_title=metric, font_family="Computer Modern")
    fig.update_xaxes(categoryorder='array', categoryarray=labels)
    fig.show()

    if export:
        fig.write_image(f'../graphics/{metric}.png')

for metric in metrics:
    boxplot(df_final, metric, labels=labels, export=True)



NameError: name 'df_final' is not defined

In [33]:
## Boxplot for Channels

def boxplot_channels(df: pd.DataFrame, export=False) -> None:
    colors = ['#808080', '#DF2F27', '#31DF41', '#334AF9']
    fig = go.Figure()
    for col, metric in zip(colors, label_channels):
        fig.add_trace(go.Box(y=df['value'][df.metric == metric],
            name=metric, marker_color=col, boxmean=True, showlegend=False, boxpoints='all'
        ))
    fig.update_layout(template='simple_white', yaxis_title='SSIM', font_family="Computer Modern")
    fig.update_xaxes(categoryorder='array')
    fig.show()

    if export:
        fig.write_image(f'../graphics/channels.png')

boxplot_channels(df_channels, True)